# NOTAM_to_Spatial 

A Python 3 Script to extract coordinates and radius (if present) from a TFR list and turn them into GeoJSON

In [1]:
import pandas as pd
import re
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString

ZDV : AS4 NM RADIUS, ZHU FIREFIGHTING <-are no wild fire related TFR but NTL DEFENCE AIRSPACE, ZKC no FIRE FIGHTING,


In [2]:
##specify file location and name
#3 letter location indicator, reused within all created files
#possible values: ZAB ZDV ZFW ZHU ZKC ZLA ZLC ZMP ZOA ZSE
tfr =r"ZDV"
#path and filename
path=r"D:\UNIGIS\MASTER\CreatedData\TFR\\"
tfr_list = tfr + r"_2021_All_Aug-Oct_revisited.xlsx"

#read the Excel Workbook
df = pd.read_excel(path + tfr_list)

In [3]:
df

,Source,Location,NOTAM #,Class,Issue Date (UTC),Effective Date (UTC),Cancel Date (UTC),Expiration Date (UTC),NOTAM Condition or LTA Subject
0,ZDV_2021-08-01,ZDV,1/6758,Airspace,07/10/2021 0335,07/10/2021 1400,08/16/2021 2148,09/10/2021 0500,!FDC 1/6758 ZDV CO..AIRSPACE 13NM N OF STEAMBO...
1,ZDV_2021-08-05,ZDV,1/1635,Airspace,08/05/2021 0233,08/05/2021 1400,08/07/2021 0427,09/30/2021 0400,!FDC 1/1635 ZDV MT..AIRSPACE 31NM SW OF BROADU...
2,ZDV_2021-08-05,ZDV,1/1585,Airspace,08/05/2021 0019,08/05/2021 0015,NaN,08/05/2021 0400,!FDC 1/1585 ZDV CANCELLED BY FDC 1/1659 ON 08/...
3,ZDV_2021-08-06,ZDV,1/2974,Airspace,08/06/2021 1847,08/06/2021 1830,NaN,08/07/2021 0400,!FDC 1/2974 ZDV CANCELLED BY FDC 1/3037 ON 08/...
4,ZDV_2021-08-07,ZDV,1/3132,Airspace,08/07/2021 1630,08/07/2021 1625,NaN,08/08/2021 0400,!FDC 1/3132 ZDV CANCELLED BY FDC 1/3192 ON 08/...
5,ZDV_2021-08-07,ZDV,1/3161,Airspace,08/07/2021 2020,08/07/2021 2040,NaN,08/08/2021 0400,!FDC 1/3161 ZDV CANCELLED BY FDC 1/3193 ON 08/...
6,ZDV_2021-08-07,ZDV,1/3169,Airspace,08/07/2021 2237,08/07/2021 2250,NaN,08/08/2021 0300,!FDC 1/3169 ZDV CANCELLED BY FDC 1/3189 ON 08/...
7,ZDV_2021-08-07,ZDV,1/3125,Airspace,08/07/2021 1436,08/07/2021 1450,08/07/2021 2018,08/08/2021 0300,!FDC 1/3125 ZDV NE..AIRSPACE 24NM SSE SCOTTSBL...
8,ZDV_2021-08-08,ZDV,1/3190,Airspace,08/08/2021 0307,08/08/2021 1300,NaN,08/11/2021 0200,!FDC 1/3190 ZDV CANCELLED BY FDC 1/4844 ON 08/...
9,ZDV_2021-08-08,ZDV,1/3198,Airspace,08/08/2021 0418,08/08/2021 1300,08/09/2021 2145,08/27/2021 0400,!FDC 1/3198 ZDV NE..AIRSPACE 24NM SSE SCOTTSBL...


In [4]:
def get_coordinates(row):
#     # find all substrings with 6 digits before N and 7 digits plus W = 8
    sub_coords = "\w{6}N\w{8}"
    coordinates = re.findall(sub_coords,row["NOTAM Condition or LTA Subject"])
    coords_list = []
    for coord in coordinates:
        #replaces old coordinates = row["NOTAM Condition or LTA Subject"].findall(sub_coords)
        #north
        deg_lat = coord[:2]
        min_lat = coord[2:4]
        sec_lat = coord[4:6]
        dd_lat = float(deg_lat) + float(min_lat)/60 + float(sec_lat)/(60*60)
        
        #west
        deg_lon = coord[7:10]
        min_lon = coord[10:12]
        sec_lon = coord[12:14]
        dd_lon = -1*(float(deg_lon) + float(min_lon)/60 + float(sec_lon)/(60*60))
        
        coords_list.append((dd_lon,dd_lat))
        
    return coords_list
        
df["Coordinates"]=df.apply(get_coordinates,axis=1)

In [5]:
df["Coordinates"]

0             [(-106.8236111111111, 40.72694444444445)]
1            [(-105.97416666666666, 45.05638888888888)]
2            [(-105.97416666666666, 45.05638888888888)]
3           [(-103.39472222222223, 41.509166666666665)]
4            [(-103.50277777777778, 43.84916666666667)]
5           [(-103.39472222222223, 41.509166666666665)]
6                      [(-107.9525, 37.30361111111111)]
7           [(-103.39472222222223, 41.509166666666665)]
8                      [(-107.9525, 37.30361111111111)]
9           [(-103.39444444444445, 41.509166666666665)]
10           [(-107.04722222222222, 36.94611111111111)]
11           [(-103.93333333333334, 43.48888888888889)]
12          [(-103.51666666666667, 43.850833333333334)]
13           [(-107.04722222222222, 36.94611111111111)]
14           [(-109.97583333333334, 39.46027777777778)]
15           [(-109.97583333333334, 39.46027777777778)]
16           [(-103.59583333333333, 44.32222222222222)]
17           [(-103.59583333333333, 44.322222222

In [6]:
def get_geometry(row):
    #transform float coordinates from get_coordinates(row)
    coords = row["Coordinates"]
    #find points
    if len(coords)==1:
        geom = Point(coords[0][0],coords[0][1])
    elif len(coords)==2:
        geom = LineString(coords)
    elif len(coords)>2:
        geom = Polygon(coords) 
    else:
        geom = None
        
    return geom
df["geometry"] = df.apply(get_geometry,axis=1)

D:\anaconda3\envs\master_env\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [7]:
df["geometry"]

0          POINT (-106.8236111111111 40.72694444444445)
1         POINT (-105.97416666666666 45.05638888888888)
2         POINT (-105.97416666666666 45.05638888888888)
3        POINT (-103.39472222222223 41.509166666666665)
4         POINT (-103.50277777777778 43.84916666666667)
5        POINT (-103.39472222222223 41.509166666666665)
6                   POINT (-107.9525 37.30361111111111)
7        POINT (-103.39472222222223 41.509166666666665)
8                   POINT (-107.9525 37.30361111111111)
9        POINT (-103.39444444444445 41.509166666666665)
10        POINT (-107.04722222222222 36.94611111111111)
11        POINT (-103.93333333333334 43.48888888888889)
12       POINT (-103.51666666666667 43.850833333333334)
13        POINT (-107.04722222222222 36.94611111111111)
14        POINT (-109.97583333333334 39.46027777777778)
15        POINT (-109.97583333333334 39.46027777777778)
16        POINT (-103.59583333333333 44.32222222222222)
17        POINT (-103.59583333333333 44.32222222

In [60]:
def get_radius(row):
    #find radius
    sub_pt_radius = "\w{1,}\\.\w{1,}NM RADIUS" ## if fraction like 1.5NM RADIUS is given
    sub_radius = "\w{1,}NM\sRADIUS"
    sub_wr_radius = "\w{1,}NM\nRADIUS"
    sub_spNM_radius = "\w{1,} NM RADIUS"
    
    #check for decimal fraction, needs to be done first, otherwise 1.5NM would result in 5NM
    radius = re.findall(sub_pt_radius,row["NOTAM Condition or LTA Subject"])
    
    #when there is no decimal fraction, look for most common case
    if len(radius)==0:
        radius = re.findall(sub_radius,row["NOTAM Condition or LTA Subject"])
    else:
        radius = radius     
    
    #when there is a line wrap btn NM and radius, its len is still 0 instead of 1:
    if len(radius)==0:
        radius = re.findall(sub_wr_radius,row["NOTAM Condition or LTA Subject"])    
    else:
        radius = radius
    
    #when there is a space btn number and NM, its len is still 0 instead of 1:
    if len(radius)==0:
        radius = re.findall(sub_spNM_radius,row["NOTAM Condition or LTA Subject"])      
    else:
        radius = radius
    
    #if len(radius)!=0:
        #if str(radius[0]).isdigit()==False:
        #if not all([str(i).isdigit() for i in radius]):
        #if not all(chr.isdigit() for chr in radius[0]):
           # radius = "NoNumber"
        #else:
           #radius = radius
        
    #so far it is 5NM RADIUS or 12NM RADIUS or 2 NM RADIUS or 1.5NM RADIUS, so cut down to the numbers
    for chars in radius:
        #one digit
        if len(chars)==10:
            radius = chars[0]
        #two digits
        elif len(chars)==11: 
            radius = chars[:2]
        #fraction
        elif len(chars)==12: 
            radius = chars[:3]
        #fraction and tens (not known if any)
        elif len(chars)==13: 
            radius = chars[:4]
        #everything else is erroneous:
        else:
            radius = "NotParsable"
        
        if radius.isdigit()==False:
            radius = radius+"isNoNumber"
        else:
            radius = radius        

    return radius
df["Radius"]=df.apply(get_radius,axis=1)

In [61]:
print(df["Radius"])

0                  8
1                  7
2                  7
3                  7
4                  3
5                 10
6                  7
7                  7
8                  5
9                 10
10                 5
11                 7
12                 3
13                 5
14                 5
15                 5
16                 5
17                 5
18                 3
19                 5
20                30
21                 3
22                 5
23                 7
24                 7
25                 7
26                 7
27                 5
28                 7
29                []
30                 5
31                 4
32                 5
33                 4
34    AS4 isNoNumber
35                 7
36                []
Name: Radius, dtype: object


In [10]:
#with Radius being a list that causes issues during JSON export,it needs to get changed:
df["Radius"] =df["Radius"].astype('string')

In [11]:
#single numbers are needed as radius instead of list residuals
def convert_radius(row):
    radius = row["Radius"]
    if radius == "[]":
        radius = 0
    return radius
df["Radius"]=df.apply(convert_radius,axis=1)

In [12]:
#string has still [] as values which cannot be converted to number format, so

df["Radius"] =df["Radius"].astype('float')

ValueError: could not convert string to float: 'AS4 '

In [ ]:
#turn radius from NM to m for buffer
def radius_to_m(row):
    radius_m = row["Radius"]*1852
    return radius_m
df["Radius_m"]=df.apply(radius_to_m,axis=1)

In [ ]:
#look at ALL rows
pd.set_option('display.max_rows', None)
print(df["Radius_m"])
#print(df.loc[174:233,"Radius"])

In [ ]:
#convert the date columns into date format
#df["Issue Date (UTC)"] = pd.to_datetime(df["Issue Date (UTC)"], errors='coerce')
# --> still text in GIS

In [ ]:
df["Issue Date (UTC)"]

According to National Wildfire Coordination Group (2018, p. 106), Keyphrase for aerial firefighting is
TO PROVIDE A SAFE ENVIRONMENT FOR WILDLAND FIRE FIGHTING AVIATION OPERATIONS. PURSUANT TO 14 CFR SECTION 91.137(A)(2) TEMPORARY FLIGHT RESTRICTIONS ARE IN EFFECT.
So it is decided to serch for variations of the term "FIRE FIGHTING" to identify relevant TFRs.

In [ ]:
#get reason of the TFR / whether it was firefighting
def get_reason(row):
#     # find all substrings with FIRE FIGHTING
    keyword = "FIRE FIGHTING"
    wr_keyword = "FIRE\nFIGHTING"
    single_keyword = "FIREFIGHTING"
    
    #find most common case
    reason = re.findall(keyword,row["NOTAM Condition or LTA Subject"])
    
    #find with linewrap
    if len(reason)==0:
        reason = re.findall(wr_keyword,row["NOTAM Condition or LTA Subject"])
    else:
        reason = reason 
    
    #find keyword written a one word
    if len(reason)==0:
        reason = re.findall(single_keyword,row["NOTAM Condition or LTA Subject"])
    else:
        reason = reason
    
    return reason
    
        
df["Reason"]=df.apply(get_reason,axis=1)

In [ ]:
df["Reason"]

For some of the NOTAMs, the journey ends with the following step. Those where firefighting ist not the reason aor where no geometry could be parsed are rejected. They are stored to Excel Workbooks to enable for manual review whether the above parsing was sufficient.

In [ ]:
# prepare dataframe for manual sanity checks in Excel
#"Reason" contains still lists so .str.len() checks for content/emptiness
df_no_fire = df[(df["Reason"].str.len() == 0)]
df_no_fire.to_excel(tfr+"_no_fire.xlsx")

The reason to purge the non-wildfire TFRs this late is, that it might be of interest to relate them to hotspot clusters as well (in a further research).

In [ ]:
# dataframe to proceed with: "Reason" shall not be empty
df = df[(df["Reason"].str.len() != 0)]

Then it is time to create a geodataframe from the dataframe containing only fire fighting related TFRs.

In [ ]:
gdf = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry=df["geometry"])

In [ ]:
# prepare geodataframe for manual sanity checks in Excel
gdf_no_geom = gdf[gdf["geometry"]==None]
gdf_no_geom.to_excel(tfr+"_no_geom.xlsx")

In [ ]:
# geodataframe to proceed with: Only with geometry
gdf = gdf[gdf["geometry"]!=None]

In [ ]:
gdf

In [ ]:
#excel check
gdf.to_excel(tfr+"_excel_check_fire_TFRs.xlsx")

List columns "Coordinates" and "Reason" are no longer needed (and would only disturn GeoJSON creation) and become omitted.

In [ ]:
#geodataframe shall use these columns
gdf= gdf[["Source","Location", "NOTAM #", "Issue Date (UTC)", "Effective Date (UTC)", "Cancel Date (UTC)", "Expiration Date (UTC)", "NOTAM Condition or LTA Subject","Radius","Radius_m","geometry"]]

In [ ]:
#prepare buffer
gdf_buffered = gdf.copy()
gdf_buffered = gdf_buffered.to_crs("EPSG:2163")

In [ ]:
#do buffer by radius: SHOULD be 0 for polygons, do it for all like this is FAST
gdf_buffered["geometry"] = gdf_buffered.buffer(gdf["Radius_m"], resolution=16)

In [ ]:
gdf["geometry"]

In [ ]:
#create geojson of gdf, points and polygons
#gdf.to_file(filename='gdf_first.geojson', driver='GeoJSON')

In [ ]:
#create geojson of gdf_buffered, just polygons
#back to WGS84 for geojson creation
gdf_buffered = gdf_buffered.to_crs("EPSG:4326")
gdf_buffered.to_file(filename= tfr+'_fire_TFRs.geojson', driver='GeoJSON')

In [ ]:
#shape file of course only works with uniform geometries
#gdf_buffered.to_file('tfr+'fire_TFRs.shp', driver='ESRI Shapefile')

In [ ]:
#plot result into new window
%matplotlib qt
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world = world.to_crs("EPSG:4326")
ax = world[world.continent == 'North America'].plot(
    color='white', edgecolor='black')
gdf_buffered.plot(ax=ax, color='red')